In [1]:
#Input the filename here
filepath="yoga mat.csv"

In [3]:
#DO NOT TOUCH THIS CELL
#Importing the required libraries
import numpy as np 
import pandas as pd 
from termcolor import colored
color="blue"
#importing the data 
data=pd.read_csv(filepath,delimiter=";")
if len(data.columns)<20:
    data=pd.read_csv(filepath)
data.head()

#Sorting according to descending order of review count
data=data.sort_values(by=['Review Count'],ascending=True)
data.head(5)

#converting data into required data types
data.Revenue=data.Revenue.apply(lambda x: str(x).replace(",","")).astype("float64")
data["FBA Fees"]=data["FBA Fees"].apply(lambda x: str(x).replace("₹","")).astype("float64")
data["Sales"]=data["Sales"].apply(lambda x: str(x).replace(",","")).astype("float64")
data["Price"]=data["Price"].apply(lambda x: str(x).replace("₹","").replace(",","")).astype("float64")

data.describe()

#checking for null values
data.isnull().sum()

data.columns

#seems like there are a lot of null values in some of the columns, and they give us no data, so removing that data an
#filling the rest with median.
ddata=data[['Product Details', 'ASIN', 'Brand', 'Price', 'Sales', 'Revenue', 'BSR',"FBA Fees",
        'Active Sellers #', 'Ratings', 'Review Count', 'Images',
       'Review velocity', 'Category', 'Delivery', 'Creation Date']]
ddata=ddata.fillna(ddata.median())

np.quantile(ddata.Revenue,0.9)

import plotly.express as px
px.box(ddata.Revenue)

#this will remove all the outliers using the box plot method 
def remove_outliers(series):
    iqr=np.quantile(series,0.75)-np.quantile(series,0.25)
    upperline=np.quantile(series,0.75)+1.5*iqr
    lowerline=np.quantile(series,0.75)-1.5*iqr
    return upperline,max(0,lowerline)

remove_outliers(ddata.Revenue)

#let's remove outliers in Revenue and Review count
ddata=ddata[(ddata.Revenue<remove_outliers(ddata.Revenue)[0]) & (ddata.Revenue>remove_outliers(ddata.Revenue)[1])]
ddata=ddata[(ddata["Review Count"]<remove_outliers(ddata["Review Count"])[0]) & (ddata["Review Count"]>remove_outliers(ddata["Review Count"])[1])]

ddata.shape

#lets look at the statistics of the data without outliers
ddata.describe()

#let's get the market share distribution vizualisation
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=2,specs=[[{"type": "pie"}, {"type": "pie"}]])
fig.add_trace(
    go.Pie(values=list(data.fillna(data.median()).Revenue),
    labels=data.fillna(data.median()).Brand,
    title="Revenue Market Distribution including Outliers"),
    row=1,col=1)
fig.add_trace(
    go.Pie(values=ddata.Revenue, labels=ddata.Brand,title="Revenue Market Distribution excluding Outliers"),
    row=1, col=2
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

#let's get the market share distribution vizualisation
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=2,specs=[[{"type": "pie"}, {"type": "pie"}]])
fig.add_trace(
    go.Pie(values=list(data.fillna(data.median()).Sales),
    labels=data.fillna(data.median()).Brand,
    title="Sales Market Distribution including Outliers"),
    row=1,col=1)
fig.add_trace(
    go.Pie(values=ddata.Sales, labels=ddata.Brand,title="Sales Market Distribution excluding Outliers"),
    row=1, col=2
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

print("Market Share of Generic Brands in distribution with outliers =",
      colored(100*data[data.Brand=="Generic"].Revenue.sum()/data.Revenue.sum(),color),"%")

print("Market Share of Generic Brands in distribution without outliers =",
      colored(100*ddata[ddata.Brand=="Generic"].Revenue.sum()/ddata.Revenue.sum(),color),"%")

print("Median revenue per seller for 30 days =",colored(ddata.Revenue.median(),color))

# Let's check how the revenue of sellers with less reviews are doing, for that let's look at the 25% percentile
np.percentile(ddata["Review Count"],25)

#Let's check how the sellers below 25 percentile are doing 
print("Median sales for products below 25 percentile Review count =",
      colored(ddata[ddata["Review Count"]<np.percentile(ddata["Review Count"],25)].Sales.median(),color))
print("Median Revenue for products below 25 percentile Review count =",
      colored(ddata[ddata["Review Count"]<np.percentile(ddata["Review Count"],25)].Revenue.median(),color))

#median Revenue of the first quartile Revenue
print("Median sales of products below 25 percentile Revenue =",
      colored(ddata[ddata["Revenue"]<np.percentile(ddata["Revenue"],25)].Sales.median(),color))
print("Median Revenue of products below 25 percentile Revenue =",
      colored(ddata[ddata["Revenue"]<np.percentile(ddata["Revenue"],25)].Revenue.median(),color))

px.bar(ddata.groupby("Brand").sum().Revenue.sort_values()[0:10],title="Revenues of Least 10 Brands")

print("The median price of the product =",colored(ddata.Price.median(),color))
print("The median FBA fees of the product =",colored(ddata["FBA Fees"].median(),color))
print("% of FBA cost =",colored(100*ddata["FBA Fees"].median()/ddata.Price.median(),color))

print("Median Weight of the product =",colored(data.Weight.dropna().median(),color))

print("Median Review Count of the product =",colored(ddata["Review Count"].median(),color))

#Finding the median dimensions of the product
dimdf=pd.DataFrame(list(data.Dimensions.str.split("x").dropna()),columns=["l","w","h"])
for x in dimdf.columns:
    dimdf[x]=dimdf[x].astype("float32")  
dimdf.median()
print("Median dimensions of product =",colored(dimdf.median()[0],color),",",
      colored(dimdf.median()[1],color),",",colored(dimdf.median()[2],color))





Market Share of Generic Brands in distribution with outliers = 0.4502000490026523 %
Market Share of Generic Brands in distribution without outliers = 0.8165156858134411 %
Median revenue per seller for 30 days = 313701.0
Median sales for products below 25 percentile Review count = 264.0
Median Revenue for products below 25 percentile Review count = 174781.5
Median sales of products below 25 percentile Revenue = 58.0
Median Revenue of products below 25 percentile Revenue = 18022.0
The median price of the product = 472.0
The median FBA fees of the product = 221.73
% of FBA cost = 46.976694915254235
Median Weight of the product = 1.1
Median Review Count of the product = 128.75
Median dimensions of product = 3.9 , 19.7 , 2.4
